In [1]:
import category_encoders as ce
import pandas as pd
import numpy as np

In [2]:
propiedadesTrain = pd.read_csv('train.csv')
propiedadesTest = pd.read_csv('test.csv')
featuresTrain = pd.read_csv('featuresTrain2.csv')
featuresTest = pd.read_csv('featuresTest2.csv')

# Este notebook lo voy a usar para hacer un Catboost de las variables categoricas:

In [3]:
# esta funcion la definimos en el TP1 para crear una columna categorica llamada caracteristicas,funciona asi:
# si una publicacion x tiene en 1 las columnas piscina y gimnasio,
# pero en escuelas cerca, centros comerciales cerca y usos multiples guarda 0
#la comuna caracteristicas guarda _gimnasio_piscina,
# si tiene 0 e todas guarda _sin_caracteristicas_
def ponerNombreCategoria(x,nombre):
    if x == 1.0:
        return nombre
    else:
        return ''
    
def ponerSinCaracteristicas(x):
    if x =='':
        return '_sin_caracteristicas_'
    else:
        return x
def crearCategorias(df_propiedades):
    df_propiedades['gimnasio_cat'] = df_propiedades['gimnasio'].apply(lambda x: ponerNombreCategoria(x,'_gimnasio_'))
    df_propiedades['piscina_cat'] = df_propiedades['piscina'].apply(lambda x: ponerNombreCategoria(x,'_piscina_'))
    df_propiedades['usosmultiples_cat'] = df_propiedades['usosmultiples']\
                            .apply(lambda x: ponerNombreCategoria(x,'_usosmultiples_'))
    df_propiedades['escuelascercanas_cat'] = df_propiedades['escuelascercanas']\
                                            .apply(lambda x: ponerNombreCategoria(x,'_escuelascercanas_'))
    df_propiedades['centroscomercialescercanos_cat'] = df_propiedades['centroscomercialescercanos']\
                                            .apply(lambda x: ponerNombreCategoria(x,'_centroscomercialescercanos_'))
    df_propiedades = df_propiedades.drop(['piscina','escuelascercanas','centroscomercialescercanos','gimnasio',\
                                         'usosmultiples'],axis=1)
    df_propiedades['caracteristicas'] = df_propiedades['gimnasio_cat'] + df_propiedades['piscina_cat']+\
                                      df_propiedades['usosmultiples_cat'] + df_propiedades['escuelascercanas_cat']+\
                                      df_propiedades['centroscomercialescercanos_cat']
    df_propiedades['caracteristicas'] = df_propiedades['caracteristicas']\
                                            .apply(lambda x: ponerSinCaracteristicas(x))
    df_propiedades = df_propiedades.drop(['gimnasio_cat','piscina_cat','usosmultiples_cat',\
                                'escuelascercanas_cat','centroscomercialescercanos_cat'],axis=1)
    return df_propiedades

In [4]:
featuresTrain = crearCategorias(featuresTrain)
featuresTest = crearCategorias(featuresTest)

In [5]:
#variables a encodear
categorias = ['tipodepropiedad','caracteristicas','provincia']

In [6]:
#featuresTrain sin lavariable encodeada:
featuresTrain.head()

,id,tipodepropiedad,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,...,lng,precio,anio,mes,dia,datoBienCargado,peso_IDF_descripcion,peso_IDF_titulo,metrosdescubiertos,caracteristicas
0,254099,Apartamento,Benito Juárez,Distrito Federal,2.210044,2.0,1.0,2.0,4.394449,4.394449,...,-99.509221,2273000.0,2015.0,8.0,23.0,1,0.086408,0.000000,0.000000,_sin_caracteristicas_
1,53461,Casa en condominio,La Magdalena Contreras,Distrito Federal,2.397895,3.0,2.0,2.0,5.594711,5.198497,...,-99.227655,3600000.0,2013.0,6.0,28.0,0,0.344349,0.165371,0.000000,_escuelascercanas__centroscomercialescercanos_
2,247984,Casa,Tonalá,Jalisco,1.791759,3.0,2.0,2.0,4.976734,5.117994,...,-99.509221,1200000.0,2015.0,10.0,17.0,1,0.992218,0.033601,3.135494,_sin_caracteristicas_
3,209067,Casa,Zinacantepec,Edo. de México,0.693147,2.0,1.0,1.0,4.158883,4.219508,...,-99.688015,650000.0,2012.0,3.0,9.0,1,0.107927,0.033601,1.609438,_escuelascercanas__centroscomercialescercanos_
4,185997,Apartamento,Zapopan,Jalisco,2.397895,2.0,1.0,1.0,4.564348,4.564348,...,-99.509221,1150000.0,2016.0,6.0,7.0,1,0.078086,0.000000,0.000000,_sin_caracteristicas_


In [7]:
#realizo el catboost encoding:
target_enc = ce.CatBoostEncoder()
target_enc.fit(featuresTrain[categorias], featuresTrain['datoBienCargado'])
featuresTrain = featuresTrain.join(target_enc.transform(featuresTrain[categorias]).add_suffix('_cb'))

In [8]:
#features Train con la variable encodeada:
featuresTrain.head()

,id,tipodepropiedad,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,...,mes,dia,datoBienCargado,peso_IDF_descripcion,peso_IDF_titulo,metrosdescubiertos,caracteristicas,tipodepropiedad_cb,caracteristicas_cb,provincia_cb
0,254099,Apartamento,Benito Juárez,Distrito Federal,2.210044,2.0,1.0,2.0,4.394449,4.394449,...,8.0,23.0,1,0.086408,0.000000,0.000000,_sin_caracteristicas_,0.478679,0.390827,0.413880
1,53461,Casa en condominio,La Magdalena Contreras,Distrito Federal,2.397895,3.0,2.0,2.0,5.594711,5.198497,...,6.0,28.0,0,0.344349,0.165371,0.000000,_escuelascercanas__centroscomercialescercanos_,0.386072,0.475439,0.413880
2,247984,Casa,Tonalá,Jalisco,1.791759,3.0,2.0,2.0,4.976734,5.117994,...,10.0,17.0,1,0.992218,0.033601,3.135494,_sin_caracteristicas_,0.426561,0.390827,0.407525
3,209067,Casa,Zinacantepec,Edo. de México,0.693147,2.0,1.0,1.0,4.158883,4.219508,...,3.0,9.0,1,0.107927,0.033601,1.609438,_escuelascercanas__centroscomercialescercanos_,0.426561,0.475439,0.420146
4,185997,Apartamento,Zapopan,Jalisco,2.397895,2.0,1.0,1.0,4.564348,4.564348,...,6.0,7.0,1,0.078086,0.000000,0.000000,_sin_caracteristicas_,0.478679,0.390827,0.407525


# Repito para featuresTest:

In [9]:
target_enc = ce.CatBoostEncoder()
target_enc.fit(featuresTest[categorias], featuresTest['datoBienCargado'])
featuresTest = featuresTest.join(target_enc.transform(featuresTest[categorias]).add_suffix('_cb'))

In [10]:
#features Train con la variable encodeada:
featuresTest.head()

,id,tipodepropiedad,ciudad,provincia,antiguedad,habitaciones,garages,banos,metroscubiertos,metrostotales,...,mes,dia,datoBienCargado,peso_IDF_descripcion,peso_IDF_titulo,metrosdescubiertos,caracteristicas,tipodepropiedad_cb,caracteristicas_cb,provincia_cb
0,4941,Casa,Miguel Hidalgo,Distrito Federal,3.401197,3.0,1.546874,4.0,5.707110,5.180463,...,7.0,20.0,0,0.018539,0.033601,3.031614,_sin_caracteristicas_,0.426768,0.392836,0.404545
1,51775,Apartamento,Mérida,Yucatán,2.210044,1.0,1.000000,1.0,4.219508,4.219508,...,10.0,24.0,1,0.714877,0.000000,0.000000,_sin_caracteristicas_,0.480004,0.392836,0.567188
2,115253,Apartamento,Coyoacán,Distrito Federal,0.000000,2.0,1.000000,2.0,4.477337,4.615121,...,5.0,30.0,1,0.530057,0.638628,2.639057,_centroscomercialescercanos_,0.480004,0.470865,0.404545
3,299321,Apartamento,Acapulco de Juárez,Guerrero,1.098612,2.0,2.000000,2.0,4.465908,4.465908,...,4.0,2.0,1,0.296724,0.091081,0.000000,_sin_caracteristicas_,0.480004,0.392836,0.364473
4,173570,Casa,Tultitlán,Edo. de México,2.397895,2.0,1.000000,1.0,4.394449,4.343805,...,8.0,15.0,0,0.619535,0.033601,0.000000,_escuelascercanas__centroscomercialescercanos_,0.426768,0.475463,0.422059


In [11]:
featuresTest = featuresTest.drop(['tipodepropiedad','caracteristicas'],axis=1)

In [12]:
featuresTrain = featuresTrain.drop(['tipodepropiedad','caracteristicas'],axis=1)

In [13]:
featuresTrain['piscina'] = propiedadesTrain['piscina']
featuresTrain['gimnasio'] = propiedadesTrain['gimnasio']
featuresTrain['usosmultiples'] = propiedadesTrain['usosmultiples']
featuresTrain['escuelascercanas'] = propiedadesTrain['escuelascercanas']
featuresTrain['centroscomercialescercanos'] = propiedadesTrain['centroscomercialescercanos']

In [14]:
featuresTest['piscina'] = propiedadesTest['piscina']
featuresTest['gimnasio'] = propiedadesTest['gimnasio']
featuresTest['usosmultiples'] = propiedadesTest['usosmultiples']
featuresTest['escuelascercanas'] = propiedadesTest['escuelascercanas']
featuresTest['centroscomercialescercanos'] = propiedadesTest['centroscomercialescercanos']

In [15]:
# transformo los dataframes en .csv:
featuresTest.to_csv('featuresTest3.csv',index=False)
featuresTrain.to_csv('featuresTrain3.csv',index=False)